In [2]:
import numpy as np
import tensorflow as tf
import json
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
with open('First Thousand Jan.json', 'r') as f:   
    sequences = json.load(f)
    sequences = sequences['Sequences']

In [4]:
s_sequences = [s['S Unit Sequence'] for s in sequences if s['S Unit Sequence'] is not None]
max_length = 0
min_length = 1000000
for s in s_sequences:
    max_length = max(max_length, len(s))
    min_length = min(min_length, len(s))
print(max_length)
print(min_length)

3831
3801


In [5]:
# Set seed for random so it's repeatable
random.seed(111)
rand_length = random.randint(min_length, max_length)

num_new_sequences = len(s_sequences)
new_sequences = []
# intToChar = {1:'A', 2:'T', 3:'C', 4:'G', 5:'N'}
for i in range(num_new_sequences):
    new_sequence = ''
    for j in range(rand_length):
        new_sequence += str(random.randint(1, 5))
    new_sequences.append(new_sequence)

In [6]:
charToInt = {'A': '1', 'T': '2', 'C': '3', 'G': '4', 'N': '5'}
int_s_sequences = []
for seq in s_sequences:
    currIntStr = ''
    for s in seq:
        currIntStr += charToInt[s]
    int_s_sequences.append(currIntStr)

In [7]:
all_sequences = int_s_sequences + new_sequences
all_sequences_labels = [1]  * len(s_sequences) + [0] * len(new_sequences)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_sequences, all_sequences_labels, test_size=0.2, random_state=111, shuffle=True)
X_train = [[[int(i) for i in x]] for x in X_train]
X_test = [[[int(i) for i in x]] for x in X_test]
y_train = [[x] for x in y_train]
y_test = [[x] for x in y_test]

# X_test = [[[x]] for x in X_test]
# y_train = [[[x]] for x in y_train]
# y_test = [[[x]] for x in y_test]

# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)



# X_train = tf.ragged.constant(X_train)
# X_test = tf.ragged.constant(X_test)
# y_train = tf.ragged.constant(y_train)
# y_test = tf.ragged.constant(y_test)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_train.bounding_shape()[-1])

AttributeError: 'list' object has no attribute 'shape'

In [10]:
model = tf.keras.Sequential([
    # Input Layer with shape = [Any,  maximum sequence length]                      
    tf.keras.layers.Input(shape=[None, X_train.bounding_shape()[-1]], batch_size=384, dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='relu')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.summary()

AttributeError: 'list' object has no attribute 'bounding_shape'

In [11]:
history = model.fit(X_train, y_train, epochs=10)

NameError: name 'model' is not defined

# Working Example
#### Below is a working example a ragged tensor input into an LSTM node than outputs a set of numbers
I am unsure of what the differences are and how to improve them

In [29]:
# xx should be 3d for LSTM
xx = tf.ragged.constant(X_train[:320])

"""
Labels represented as OneHotEncoding so you 
should use CategoricalCrossentropy instade of SparseCategoricalCrossentropy
"""

yy = np.array(y_train[:320])

In [11]:
print(xx.shape)
print(yy.shape)
print(xx.bounding_shape()[-1])

(320, None, None)
(320, 1)
tf.Tensor(3827, shape=(), dtype=int64)


In [12]:
print(xx[1])

<tf.RaggedTensor [[3, 5, 3, 1, 2, 2, 1, 5, 5, 5, 1, 2, 2, 3, 3, 3, 4, 5, 5, 3, 5, 1, 3, 4, 3, 1, 5, 1, 1, 1, 3, 5, 4, 5, 2, 4, 4, 4, 5, 5, 3, 2, 1, 2, 1, 5, 2, 4, 2, 1, 4, 1, 4, 5, 1, 1, 3, 5, 2, 4, 1, 3, 5, 2, 3, 5, 4, 4, 5, 4, 3, 2, 1, 1, 2, 4, 1, 5, 2, 5, 5, 2, 5, 2, 5, 1, 3, 4, 2, 5, 2, 2, 2, 2, 4, 5, 3, 2, 3, 4, 4, 1, 1, 3, 3, 3, 2, 5, 2, 3, 4, 1, 5, 5, 2, 4, 2, 3, 2, 5, 3, 3, 1, 3, 3, 2, 3, 1, 3, 4, 1, 4, 5, 3, 5, 2, 3, 4, 5, 4, 2, 2, 2, 2, 3, 3, 3, 1, 3, 3, 1, 3, 2, 2, 1, 1, 3, 4, 3, 1, 2, 4, 3, 4, 4, 5, 2, 2, 1, 2, 3, 1, 3, 4, 5, 2, 5, 2, 3, 4, 3, 5, 5, 2, 1, 5, 3, 1, 5, 2, 1, 4, 4, 3, 5, 3, 1, 3, 5, 1, 1, 2, 3, 5, 5, 4, 2, 2, 4, 2, 2, 1, 3, 5, 4, 1, 5, 2, 2, 4, 4, 4, 1, 5, 4, 5, 2, 3, 4, 5, 1, 3, 3, 2, 1, 4, 2, 4, 1, 2, 4, 1, 1, 2, 4, 2, 2, 5, 5, 1, 1, 3, 1, 5, 5, 3, 1, 1, 5, 2, 1, 1, 4, 3, 1, 5, 4, 1, 4, 3, 5, 5, 2, 3, 2, 2, 5, 5, 4, 4, 3, 1, 2, 3, 4, 4, 3, 5, 5, 3, 3, 5, 2, 4, 5, 3, 1, 3, 1, 1, 1, 5, 3, 3, 5, 2, 2, 4, 5, 5, 5, 5, 2, 5, 3, 2, 5, 1, 4, 2, 4, 3, 1, 1, 4, 2, 3, 

In [26]:
# For ragged tensor , get maximum sequence length
max_seq = xx.bounding_shape()[-1]
mdl = tf.keras.Sequential([
    # Input Layer with shape = [Any,  maximum sequence length]                      
    tf.keras.layers.Input(shape=[None, max_seq], batch_size=2, dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# CategoricalCrossentropy
mdl.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

mdl.summary()
history = mdl.fit(xx, yy, epochs=10)


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (2, 64)                   996352    
_________________________________________________________________
dense_9 (Dense)              (2, 64)                   4160      
_________________________________________________________________
dense_10 (Dense)             (2, 1)                    65        
Total params: 1,000,577
Trainable params: 1,000,577
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 2s 12ms/step - loss: 0.1876 - accuracy: 0.7719
Epoch 2/10
10/10 [==============================] - 0s 13ms/step - loss: 0.0455 - accuracy: 0.9563
Epoch 3/10
10/10 [==============================] - 0s 13ms/step - loss: 0.0207 - accuracy: 0.9969
Epoch 4/10
10/10 [==============================] - 0s 12ms/step - loss: 0.01

In [27]:
mdl.predict(xx)

array([[ 6.51379824e-02],
       [-9.53550357e-03],
       [ 1.01961458e+00],
       [ 1.01601863e+00],
       [ 5.71612157e-02],
       [ 1.01370466e+00],
       [ 1.01435876e+00],
       [ 2.99431998e-02],
       [-2.24298965e-02],
       [ 1.01687598e+00],
       [ 3.29123698e-02],
       [ 1.06289044e-01],
       [ 4.81510423e-02],
       [-2.70665926e-03],
       [ 9.76778150e-01],
       [ 1.01370466e+00],
       [ 5.71255125e-02],
       [ 1.01549375e+00],
       [ 7.60215670e-02],
       [ 1.02919328e+00],
       [ 1.06144750e+00],
       [ 1.01370466e+00],
       [ 6.93049431e-02],
       [ 1.59180332e-02],
       [ 1.03428864e+00],
       [-5.09097939e-03],
       [-2.87502003e-03],
       [ 5.96583001e-02],
       [ 7.50106052e-02],
       [ 1.01370466e+00],
       [-8.89339019e-03],
       [-5.44811003e-02],
       [ 9.20302868e-01],
       [ 1.03505814e+00],
       [ 1.01692295e+00],
       [ 1.01692295e+00],
       [ 1.00412452e+00],
       [ 1.01370466e+00],
       [-1.3

In [24]:
yy

array([[0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
    

In [30]:
x2 = tf.ragged.constant(X_train[330:])
y2 = np.array(y_train[330:])

In [31]:
mdl.predict(x2)

array([[-0.09965168],
       [-0.04884674],
       [ 0.9655971 ],
       [ 0.97677815],
       [ 1.0143588 ],
       [ 1.0342886 ],
       [ 0.02645987],
       [-0.11107711],
       [-0.05142243],
       [ 1.016923  ],
       [ 0.11276975],
       [ 1.023194  ],
       [ 0.06993449],
       [ 1.0125865 ],
       [ 0.11474872],
       [ 1.007819  ],
       [ 0.98851204],
       [ 0.25741044],
       [ 0.82136005],
       [ 1.016923  ],
       [ 1.0137047 ],
       [ 0.00359402],
       [ 1.0120919 ],
       [ 0.32251677],
       [ 1.016876  ],
       [-0.0030071 ],
       [ 1.0669916 ],
       [ 0.96022654],
       [ 1.0155317 ],
       [ 1.072702  ],
       [ 1.0137047 ],
       [ 1.0169231 ],
       [ 0.40631497],
       [ 0.11149028],
       [ 0.00507334],
       [ 1.0124109 ],
       [-0.01153294],
       [-0.0162803 ],
       [-0.03177276],
       [ 0.10675846],
       [ 1.016923  ],
       [ 0.03558903],
       [ 0.09248678],
       [-0.0916857 ],
       [ 0.2198877 ],
       [ 1

In [32]:
y2

array([[0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0]])

In [33]:
!pip install pyyaml h5py


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [36]:
mdl.save_weights('confidence_model')